In [1]:
from bs4 import BeautifulSoup
import urllib.request
import ssl

def GetKospi200():     
    stockDic = dict()
    lastPageNum = 0

    # 마지막 페이지 찾기
    base_url = "http://finance.naver.com/sise/entryJongmok.nhn?&page="
    target_url = base_url + str(1)
    context = ssl._create_unverified_context()
    # soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "lxml")
    soup = BeautifulSoup(urllib.request.urlopen(target_url, context=context).read().decode('euc-kr', 'ignore'), "lxml")
    for item in soup.find_all('td'):
        if item.has_attr('class') and 'pgRR' in item['class']:
            lastPageNum = int(str(item.a['href']).replace('/sise/entryJongmok.nhn?&page=', ''))

    for i in range(1, lastPageNum+1):
        target_url = base_url + str(i)
        # soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "lxml")
        soup = BeautifulSoup(urllib.request.urlopen(target_url, context=context).read().decode('euc-kr', 'ignore'), "lxml")
        postNoList = soup.find_all('a')


        # 종목코드와 종목명 담기
        for item in postNoList:
            if item.has_attr('target') and '_parent' in item['target'] and item.has_attr('href'):
                if str(item['href']).startswith('/item/main.nhn?code='):
                    stockDic[str(item['href']).replace('/item/main.nhn?code=', '')] = item.text
    return stockDic


In [120]:
import pandas as pd

def GetPriceData(item):    
    
    # print(item)
    code = item[0]
    name = item[1]
                
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    url = "http://finance.naver.com/item/sise_day.nhn?code=" + code
    page = "&page="
    idx = 1
#     datetimeList = []
#     closePriceList = []
#     startPriceList = []
#     minPriceList = []
#     maxPriceList = []
#     amountList = []


    bBool = True
    outputFileName = directory + code + '.csv'
    
    df = pd.read_csv(outputFileName)

    
    with open(outputFileName, 'a') as f:
        while bBool:
            fullAddr = url + page + str(idx)

            source_code = requests.get(fullAddr, headers = headers)
            if source_code is None:
                break

            soup = BeautifulSoup(source_code.text,"lxml")
            if soup.find('td', class_='on').find('a').text != str(idx):
                break

            for tr in filter(lambda x:x.get("onmouseout") is not None, soup.find_all("tr")):
                if tr.find("span",class_ = "tah p10 gray03") is None:
                    # 가격데이터가 없으면 False로 빠져나옴
                    bBool = False

                else:
                    tDate = tr.find("span",class_ = "tah p10 gray03").text
                    cPrice = tr.find_all("span",class_ = "tah p11")
                    sIdx = 1

                    if len(cPrice) != 5 :
                        sIdx = 2

                    dt = tDate.replace("." ,"-")
                    if (df["date"] == dt).sum() > 0:                        
#                         print(dt, " 가격존재 ")
                        f.close()
                        return

                    pClose = float(cPrice[0].text.replace("," ,""))
                    pStart = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    pMax   = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    pMin   = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    amount = float(cPrice[sIdx].text.replace("," ,""))

                    stockList = []
                    stockList.append(dt)
                    stockList.append(code)
                    stockList.append(pStart)
                    stockList.append(pClose)
                    stockList.append(pMin)
                    stockList.append(pMax)
                    stockList.append(amount)                
                    f.writelines(",".join([str(x) for x in stockList]) + '\n') 
            idx += 1
    f.close()
    
    # 투자정보
    # totalCnt = []  # 상장주식수
    # forignerHaveLimit = []  # 외국인한도보유주식수
    # forignerHaveCnt = []  # 외국인보유주식수
    # max52week = []  # 52주 최고
    # min52week = []  # 52주 최저
    # per = []
    # eps = []
    # per_eps_date = []
    # estimate_per = []  # 추정 PER
    # estimate_eps = []  # 추정 EPS
    # pbr = []
    # bps = []
    # pbr_bps_date = []
    # dvr = []  # 배당수익

    url2 = "https://finance.naver.com/item/main.nhn?code=" + code
    source_code = requests.get(url2, headers = headers)
    if source_code is None:
        pass
    else:
        soup = BeautifulSoup(source_code.text, "lxml")
        
        totalCnt = soup.find(id='tab_con1').find_all('em')[2].text.replace(',', '').replace('N/A', '0')
        forignerHaveLimit = soup.find(id='tab_con1').find_all('em')[5].text.replace(',', '').replace('N/A', '0')
        forignerHaveCnt = soup.find(id='tab_con1').find_all('em')[6].text.replace(',', '').replace('N/A', '0')
        max52week = soup.find(id='tab_con1').find_all('em')[10].text.replace(',', '').replace('N/A', '0')
        min52week = soup.find(id='tab_con1').find_all('em')[11].text.replace(',', '').replace('N/A', '0')
        per = soup.find(id='tab_con1').find_all('em')[12].text.replace(',', '').replace('N/A', '0')
        eps = soup.find(id='tab_con1').find_all('em')[13].text.replace(',', '').replace('N/A', '0')
        per_eps_date = soup.find(id='tab_con1').find_all('span', class_='date')[0].text.replace('(','').replace(')','') if len(soup.find(id='tab_con1').find_all('span', class_='date')) > 0 else ""
        estimate_per = soup.find(id='tab_con1').find_all('em')[14].text.replace(',', '').replace('N/A', '0')
        estimate_eps = soup.find(id='tab_con1').find_all('em')[15].text.replace(',', '').replace('N/A', '0')
        pbr = soup.find(id='tab_con1').find_all('em')[16].text.replace(',', '').replace('N/A', '0')
        bps = soup.find(id='tab_con1').find_all('em')[17].text.replace(',', '').replace('N/A', '0')
        pbr_bps_date = soup.find(id='tab_con1').find_all('span', class_='date')[1].text.replace('(','').replace(')','') if len(soup.find(id='tab_con1').find_all('span', class_='date')) > 1 else ""
        dvr = soup.find(id='tab_con1').find_all('em')[18].text.replace(',', '').replace('N/A', '0')                
        
        financeList = []
        financeList.append(datetime.today().strftime("%Y-%m-%d"))
        financeList.append(code)
        financeList.append(totalCnt)
        financeList.append(forignerHaveLimit)
        financeList.append(forignerHaveCnt)
        financeList.append(forignerHaveLimit)
        financeList.append(forignerHaveCnt)
        financeList.append(max52week)
        financeList.append(min52week)
        financeList.append(per)
        financeList.append(eps)
        financeList.append(per_eps_date)
        financeList.append(estimate_per)
        financeList.append(estimate_eps)
        financeList.append(pbr)
        financeList.append(bps)
        financeList.append(pbr_bps_date)
        financeList.append(dvr)  
        financeFileName = 'data/' + 'stock_finance_data.csv'
        with open(financeFileName, 'a') as f:
            f.writelines(",".join([str(x) for x in financeList]) + '\n') 
        f.close()

In [183]:
import os
import requests 
from datetime import datetime
import pandas as pd

def crawling():
    file_list = os.listdir('data')
    exist_list = []
    for file in file_list:
        if 'finance' not in file and '.csv' in file:
            exist_list.append(file)

    directory = 'data/'
    stockDict = GetKospi200()
    # stockDict = {'192400' : '쿠쿠홀딩스'}

    kospiListFile = "kospi_list.csv"
    bFlag = False
    if os.path.exists(directory+kospiListFile):
        bFlag = bool((pd.read_csv(directory+kospiListFile, names=["date", "code", "name"], header=None)["date"] == datetime.today().strftime("%Y-%m-%d")).sum())
    
    with open(directory + "/" + kospiListFile, 'a') as f:
        for key in stockDict.keys():
            if not bFlag:        
                l = []
                l.append(datetime.today().strftime("%Y-%m-%d"))
                l.append(key)
                l.append(stockDict[key])
                f.writelines(",".join([str(x) for x in l]) + '\n') 

            if str(key) not in exist_list:
                target = (key, stockDict[key])                
                GetPriceData(target)
    f.close()

In [184]:
crawling()

In [175]:
# 특정날짜 데이터 제거 및 reset_index
import pandas as pd
def removeData(date = "2021-02-26"):
    directory = 'data'
    file_list = os.listdir(directory)

    for file in file_list:
        if 'finance' not in file and '.csv' in file and 'kospi' not in file:   
            print(file)
            df = pd.read_csv(directory + "/" + file)
            if "Unnamed: 0" in df.columns:
                df = pd.read_csv(directory + "/" + file).drop("Unnamed: 0", axis=1)

            if (df["date"] == date).sum() > 0:
                df = df.drop(df[df["date"] == date].index)
                df.to_csv(directory + "/" + file, index=False)

In [177]:
# 특정날짜 데이터 제거 및 reset_index
import pandas as pd
def resetIndex():
#     cols = ["date", "code", "start", "close", "min", "max", "amount"]
    directory = 'data'
    file_list = os.listdir(directory)

    for file in file_list:
        if 'finance' not in file and '.csv' in file and 'kospi' not in file:     
            print(file)
            df = pd.read_csv(directory + "/" + file)
            if "Unnamed: 0" in df.columns:
                df = pd.read_csv(directory + "/" + file).drop("Unnamed: 0", axis=1)

            df = df.sort_values(by="date").reset_index().drop("index", axis=1)
            df.to_csv(directory + "/" + file, index=False)
                

In [178]:
resetIndex()

005380.csv
001680.csv
017800.csv
272210.csv
115390.csv
241590.csv
000210.csv
108670.csv
105630.csv
000990.csv
003490.csv
049770.csv
192820.csv
064350.csv
003240.csv
005180.csv
039490.csv
069260.csv
139480.csv
316140.csv
018880.csv
006260.csv
284740.csv
018260.csv
282330.csv
192400.csv
035720.csv
112610.csv
145990.csv
035250.csv
003090.csv
010950.csv
064960.csv
001120.csv
009540.csv
001450.csv
185750.csv
128940.csv
031430.csv
009150.csv
001040.csv
033780.csv
009420.csv
008930.csv
003520.csv
036570.csv
120110.csv
000270.csv
016360.csv
009240.csv
000660.csv
352820.csv
001230.csv
001740.csv
008770.csv
207940.csv
326030.csv
021240.csv
005490.csv
267250.csv
010060.csv
024110.csv
006400.csv
294870.csv
003550.csv
036460.csv
020000.csv
103140.csv
029780.csv
214320.csv
079160.csv
011790.csv
000070.csv
004000.csv
005440.csv
096770.csv
011780.csv
014820.csv
030000.csv
032830.csv
008560.csv
000100.csv
023530.csv
035420.csv
004170.csv
020560.csv
052690.csv
204320.csv
071840.csv
138930.csv
001800.csv

In [167]:
pd.read_csv("data" + "/" + "009240.csv")

,Unnamed: 0,date,code,start,close,min,max,amount
0,0,2002-07-16,9240,6020.0,6920.0,6000.0,6920.0,562319.0
1,1,2002-07-18,9240,7950.0,7950.0,7950.0,7950.0,62853.0
2,2,2002-07-19,9240,9140.0,9140.0,8830.0,9140.0,2406392.0
3,3,2002-07-22,9240,9700.0,10500.0,9500.0,10500.0,2124721.0
4,4,2002-07-23,9240,12050.0,12050.0,11650.0,12050.0,1605507.0
...,...,...,...,...,...,...,...,...
4599,4599,2021-02-19,9240,100000.0,101000.0,99000.0,101500.0,67547.0
4600,4600,2021-02-22,9240,101000.0,99200.0,97700.0,101000.0,112343.0
4601,4601,2021-02-23,9240,98700.0,100500.0,97600.0,102500.0,84469.0
4602,4602,2021-02-24,9240,100500.0,98700.0,98000.0,100500.0,72979.0
